In [205]:
import xlsxwriter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import openpyxl
from openpyxl.chart import BarChart3D,Reference
import pandas

In [206]:
EAGLERAYBP_DATA = r'O:\PDI-Barlap\Aoo\YLD_MNT\EAGLERAYBP.csv'
FDBHMR6ET_DATA = r'O:\PDI-Barlap\Aoo\YLD_MNT\FDBHMR6ET.csv'
OUT_PATH = r'O:\PDI-Barlap\Aoo\YLD_MNT\YLD.csv'
FY_Car = r'O:\PDI-Barlap\Aoo\YLD_MNT\Calendar.csv'

In [207]:
eag = pd.read_csv(EAGLERAYBP_DATA)
fdb = pd.read_csv(FDBHMR6ET_DATA)
all = pd.concat([eag,fdb])
all["STARTQTY"] = all["STARTQTY"].fillna(0)
all["REASONQTY"] = all["REASONQTY"].fillna(0)
all["BONUSQTY"] = all["BONUSQTY"].fillna(0)
all["DEFECTPERCENTAGE"] = all["DEFECTPERCENTAGE"].fillna(0)
all["NORMALISED"] = all["NORMALISED"].fillna(0)
all["TotalQty"] = all["FinalShipQty"] + all["TotalScrapQty"]
all["Net_REASONQTY"] = all["FinalShipQty"] + all["TotalScrapQty"]
all["%DEFECT"] = all["REASONQTY"]/all["TotalQty"]*100
loading_parts = (pd.pivot_table(all, values='TotalQty', index=['Time_Date'], columns=['PRODUCT'], aggfunc=np.mean,fill_value=0 , margins=False).reset_index().rename_axis(None, axis=0))
defect_parts = (pd.pivot_table(all, values='REASONQTY', index=['PRODUCT','Time_Date'], columns=['DEF_CODE'], aggfunc=np.sum, fill_value=0, margins=False).reset_index().rename_axis(None, axis=0))

In [212]:
E_Defect = defect_parts.loc[(defect_parts.PRODUCT == "EAGLERAYBP")]
E_Defect = pd.merge(loading_parts, E_Defect, on="Time_Date", how="outer").drop(columns=["FDBHMR6ET"])
E_Defect = E_Defect.rename(columns={"EAGLERAYBP": "TOTAL", 110: "STH",130: "BRK"})
E_Defect["PRODUCT"] = E_Defect["PRODUCT"].fillna("EAGLERAYBP")
E_Defect["STH"] = E_Defect["STH"].fillna(0)
E_Defect["BRK"] = E_Defect["BRK"].fillna(0)
F_Defect = defect_parts.loc[(defect_parts.PRODUCT == "FDBHMR6ET")]
F_Defect = pd.merge(loading_parts, F_Defect, on="Time_Date", how="outer").drop(columns=["EAGLERAYBP"])
F_Defect = F_Defect.rename(columns={"FDBHMR6ET": "TOTAL", 110: "STH",130: "BRK"})
F_Defect["PRODUCT"] = F_Defect["PRODUCT"].fillna("FDBHMR6ET")
F_Defect["STH"] = F_Defect["STH"].fillna(0)
F_Defect["BRK"] = F_Defect["BRK"].fillna(0)
EF_Defect = E_Defect.append(F_Defect)
EF_Defect["%DEFECT_STH"] = EF_Defect["STH"]/EF_Defect["TOTAL"]*100
EF_Defect["%DEFECT_BRK"] = EF_Defect["BRK"]/EF_Defect["TOTAL"]*100
Fiscal = pd.read_csv(FY_Car)
EF_Defect = pd.merge(Fiscal, EF_Defect, on="Time_Date", how="inner")
EF_Defect.to_csv(OUT_PATH, index=None)
EF_Defect.head(10)


,WW,Time_Date,TOTAL,PRODUCT,STH,BRK,%DEFECT_STH,%DEFECT_BRK
0,FY2201,20210703,3586,EAGLERAYBP,0.0,135.0,0.000000,3.764640
1,FY2201,20210703,1851,FDBHMR6ET,0.0,0.0,0.000000,0.000000
2,FY2201,20210704,24625,EAGLERAYBP,0.0,52.0,0.000000,0.211168
3,FY2201,20210704,5517,FDBHMR6ET,1.0,0.0,0.018126,0.000000
4,FY2201,20210705,9530,EAGLERAYBP,92.0,126.0,0.965373,1.322141
5,FY2201,20210705,6273,FDBHMR6ET,0.0,0.0,0.000000,0.000000
6,FY2201,20210706,5353,EAGLERAYBP,124.0,29.0,2.316458,0.541752
7,FY2201,20210706,3274,FDBHMR6ET,0.0,0.0,0.000000,0.000000
8,FY2201,20210707,16046,EAGLERAYBP,109.0,0.0,0.679297,0.000000
9,FY2201,20210707,1753,FDBHMR6ET,0.0,0.0,0.000000,0.000000


In [215]:
GBY = (pd.pivot_table(EF_Defect, values='TOTAL', index=['WW'], columns=['PRODUCT'], aggfunc=np.mean, fill_value=0, margins=False).reset_index().rename_axis(None, axis=0))
GBY

PRODUCT,WW,EAGLERAYBP,FDBHMR6ET
0,FY2201,10995.142857,3236.285714
1,FY2202,12034.000000,1654.857143
2,FY2203,22876.000000,2082.285714
3,FY2204,8080.000000,2867.142857
4,FY2205,8373.571429,2783.571429
5,FY2206,8274.571429,4892.000000
6,FY2207,12236.714286,2988.571429
7,FY2208,4324.000000,2875.857143
8,FY2209,7980.857143,3952.428571
9,FY2210,21500.142857,5203.714286
